# Evaluation Metrics

### Import Libraries and Dependencies

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

### Read Signal Data CSV and Import as Pandas DataFrame

In [6]:
# Read in data and set index
filepath = Path('../Resources/trading_signals.csv')
signals_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)
signals_df = signals_df.set_index("date", drop=True)
signals_df.tail(100)

,close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
date,,,,,,,,,,,,
4/30/19,200.67,188.8716,174.96570,1,0.0,500,0.0,100335.0,23725.0,124060.0,-0.015631,0.24060
5/1/19,210.52,189.6634,175.30400,1,0.0,500,0.0,105260.0,23725.0,128985.0,0.039699,0.28985
5/2/19,209.15,190.4058,175.64830,1,0.0,500,0.0,104575.0,23725.0,128300.0,-0.005311,0.28300
5/3/19,211.75,191.2196,176.08090,1,0.0,500,0.0,105875.0,23725.0,129600.0,0.010133,0.29600
5/6/19,208.48,191.9298,176.46970,1,0.0,500,0.0,104240.0,23725.0,127965.0,-0.012616,0.27965
...,...,...,...,...,...,...,...,...,...,...,...,...
9/13/19,218.75,207.0573,200.50975,1,0.0,500,0.0,109375.0,22495.0,131870.0,-0.016171,0.31870
9/16/19,219.90,207.3707,200.63715,1,0.0,500,0.0,109950.0,22495.0,132445.0,0.004360,0.32445
9/17/19,220.70,207.7843,200.79135,1,0.0,500,0.0,110350.0,22495.0,132845.0,0.003020,0.32845


### Prepare Portfolio Evaluation Metrics DataFrame

In [3]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


### Calculate and Assign Portfolio Evaluation Metrics

In [4]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,0.32975
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [5]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)
portfolio_evaluation_df

,Backtest
Annual Return,0.0666829
Cumulative Returns,0.32975
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [7]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,0.32975
Annual Volatility,0.137751
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [8]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,0.32975
Annual Volatility,0.137751
Sharpe Ratio,0.484084
Sortino Ratio,NaN


In [13]:
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
sortino_ratio_df.loc[sortino_ratio_df['Portfolio Daily Returns'] < target, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
sortino_ratio_df

,Portfolio Daily Returns,Downside Returns
date,,
9/22/14,NaN,0.000000
9/23/14,NaN,0.000000
9/24/14,0.000000,0.000000
9/25/14,0.000000,0.000000
9/26/14,0.000000,0.000000
...,...,...
9/13/19,-0.016171,0.000261
9/16/19,0.004360,0.000000
9/17/19,0.003020,0.000000


In [14]:
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,0.32975
Annual Volatility,0.137751
Sharpe Ratio,0.484084
Sortino Ratio,0.685956


### Prepare Trade Evaluation Metrics DataFrame

In [15]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


### Calculate and Append Trade Evaluation Metrics

In [16]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['close'] * row['Entry/Exit Position'])
        exit_share_price = row['close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'AAPL',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,AAPL,2/12/15,8/3/15,500.0,126.46,118.44,63230.0,59220.0,-4010.0
1,AAPL,12/9/15,1/13/16,500.0,115.62,97.39,57810.0,48695.0,-9115.0
2,AAPL,4/25/16,6/14/16,500.0,105.08,97.46,52540.0,48730.0,-3810.0
3,AAPL,8/22/16,3/26/18,500.0,108.51,172.77,54255.0,86385.0,32130.0
4,AAPL,4/19/18,11/30/18,500.0,172.80,178.58,86400.0,89290.0,2890.0
5,AAPL,3/28/19,7/22/19,500.0,188.72,207.22,94360.0,103610.0,9250.0
